In [0]:
from bs4 import BeautifulSoup as BS
from pandas import ExcelWriter
from urllib.request import urlopen
import numpy as np
import pandas as pd


def make_soup(url):
    """
    soup function which returns the html code 
    of the specific page as a soup object.
    it takes the desired url as input

    """
  thepage = urllib.request.urlopen(url)
  soupdata = BeautifulSoup(thepage, "html.parser")
  return soupdata


# List of Test States which updates the URL
state_List = ['NJ','WA','WV','WI','NY']

Pf_Dict = {}
for state in state_List:
  # This is the website with the Pf data,change as necessary
  soup = make_soup("https://publicplansdata.org/quick-facts/by-state/state/?state="+state)

  # In my case I am interested in tables,adjust as needed
  table = soup.find('table')
  # Get header object of the table
  t_h = [th.text for th in table.find_all('th')]
  # Tablerow object
  tablerows = table.find_all('tr')

  Colu = []
  Fd_list = []
  # Loop through the tablerow object
  for each_r in tablerows:
    tr_d = each_r.find_all('td')
  # Loop through the tabledata object
    er_d = [ed.text for ed in tr_d]
    Colu.append(er_d)
  # Make the Pf dataframe
  Pf_Dict[state] = pd.DataFrame(Colu)

  # Set headers
  Pf_Dict[state].columns = t_h

  # Center Headers
  pd.set_option('colheader_justify', 'center')
  # Drop None Rows
  Pf_Dict[state] = Pf_Dict[state].dropna()

  # Keep First 2 and last columns
  Pf_Dict[state] = Pf_Dict[state].iloc[:,[0,1,-1]]

  
  # Convert column to $Dollars
  Pf_Dict[state]['Assets (millions)'] = ['${:,.2f}'.format(float(x.replace(',',''))) for x in Pf_Dict[state]['Assets (millions)']]

# Dictionary of Pfs
Pf_Dict

writer = ExcelWriter(r"C:\Users\INPUT YOUR YOUR DESTINATION FOLDER\PFund.xlsx",engine ='xlsxwriter')

#Loop through dictionary,to convert each df to separate worksheet in same workbook
for Pf_k, St_Pf in Pf_Dict.items():
  Pf_Dict[Pf_k].to_excel(writer,Pf_k)
# Save the writer object
writer.save()
